## VGG

In [1]:
import d2l 
from mxnet import *
from mxnet.gluon import nn
npx.set_np()

In [6]:
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size= 3,
                         padding= 1, activation= 'relu'))
    blk.add(nn.MaxPool2D(pool_size= 2, strides= 2))
    return blk

In [7]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [8]:
def vgg(conv_arch):
    net = nn.Sequential()
    
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(nn.Dense(4096, activation= 'relu'), nn.Dropout(0.5),
           nn.Dense(4096, activation= 'relu'), nn.Dropout(0.5),
           nn.Dense(10))
    return net

net = vgg(conv_arch)

In [9]:
net.initialize()
X = np.random.uniform(size= (1, 1, 224, 224))

for blk in net:
    X = blk(X)
    print(blk.name, 'output_shape: \t', X.shape)

sequential7 output_shape: 	 (1, 64, 112, 112)
sequential8 output_shape: 	 (1, 128, 56, 56)
sequential9 output_shape: 	 (1, 256, 28, 28)
sequential10 output_shape: 	 (1, 512, 14, 14)
sequential11 output_shape: 	 (1, 512, 7, 7)
dense3 output_shape: 	 (1, 4096)
dropout2 output_shape: 	 (1, 4096)
dense4 output_shape: 	 (1, 4096)
dropout3 output_shape: 	 (1, 4096)
dense5 output_shape: 	 (1, 10)


In [10]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
small_conv_arch

[(1, 16), (1, 32), (2, 64), (2, 128), (2, 128)]

In [11]:
net = vgg(small_conv_arch)

In [12]:
lr = 0.05
num_epochs = 1
batch_size = 128

In [ ]:
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr)